In [27]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense, Dropout, Activation
from keras.optimizers import Adam

In [28]:
TB=0
TE=3389
INPUT_SIZE=7
OUTPUT_SIZE=1
BATCH_SIZE=1
TIME_STEPS=1
CELL_SIZE=7
HIDD=100

In [29]:
f=open('test.csv') 
df=pd.read_csv(f)     
data=df.iloc[:,2:10].values

In [30]:
def get_train_data(train_begin=TB,train_end=TE):
    global train_mean, train_std
    data_train=data[train_begin:train_end]
    train_mean=np.mean(data_train,axis=0)
    train_std=np.std(data_train,axis=0)
    normalized_train_data=(data_train-train_mean)/train_std
    train_x = normalized_train_data[:,:7]
    train_y = normalized_train_data[:,7,np.newaxis]
    return train_x[:, np.newaxis], train_y[:, np.newaxis]
train_x, train_y=get_train_data()

In [31]:
def get_test_data(test_begin=TE):
    global test_mean, test_std
    data_test=data[test_begin:]
    test_mean=np.mean(data_test,axis=0)
    test_std=np.std(data_test,axis=0)
    test_data=(data_test-test_mean)/test_std
    test_x=test_data[:,:7]
    test_y=test_data[:,7,np.newaxis]
    return test_x[:, np.newaxis], test_y[:, np.newaxis]
test_x, test_y=get_test_data()

In [32]:
model = Sequential()

# build a LSTM RNN
model.add(LSTM(
    batch_input_shape=(BATCH_SIZE, TIME_STEPS, INPUT_SIZE),       # Or: input_dim=INPUT_SIZE, input_length=TIME_STEPS,
    output_dim=CELL_SIZE,
    return_sequences=True,      # True: output at all steps. False: output as last step.
    stateful=True              # True: the final state of batch1 is feed into the initial state of batch2
))

model.add(Dropout(0.2))

model.add(LSTM(
    HIDD,
    return_sequences=True,
    stateful=True
))

model.add(Dropout(0.2))
# add output layer

model.add(TimeDistributed(Dense(OUTPUT_SIZE)))
#adam = Adam(LR)
model.compile(optimizer='adam',loss='mse')
train_history=model.fit(train_x, train_y, batch_size=BATCH_SIZE, epochs=501, verbose=2)

C:\Users\Administrator\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(batch_input_shape=(1, 1, 7), stateful=True, units=7, return_sequences=True)`
  


Epoch 1/501


InvalidArgumentError: The node 'time_distributed_3/bias/Assign' has inputs from different frames. The input 'time_distributed_3/Const' is in frame 'training_1/Adam/gradients/lstm_4/while/lstm_4/while/'. The input 'time_distributed_3/bias' is in frame ''.

In [ ]:
trainScore = model.evaluate(train_x, train_y, batch_size=BATCH_SIZE, verbose=1)
print(trainScore)

In [ ]:
predict=model.predict(test_x, batch_size=BATCH_SIZE,verbose=1)
print(predict)

In [ ]:
predict.reshape(-1)
np.array(predict)*test_std[3]+test_mean[3]